# EDA: Crimes Against Women in India - TAM Round 1 23BCE2086

## Overview
I had created this notebook which performs comprehensive EDA on crimes against women data, including identification of high-crime states, clustering analysis, and crime type distribution.

## 1. Data Cleaning & Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# make the plots look clean and professional
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# load the data from GitHub
github_url = "https://raw.githubusercontent.com/VK-SHRIDHARAN/23BCE2086-EDA-On-Women-Safety_TAM/main/"

try:
    # load from GitHub
    df = pd.read_csv(github_url + 'CrimesOnWomenData.csv', index_col=0)
    description = pd.read_csv(github_url + 'description.csv', index_col=0)
    print(f"Data loaded successfully from GitHub")
except:
    # fallback to local if available
    try:
        df = pd.read_csv('CrimesOnWomenData.csv', index_col=0)
        description = pd.read_csv('description.csv', index_col=0)
        print("Data loaded from local directory")
    except:
        print("Error: Could not load data from GitHub or local directory")
        raise

print("\nDataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head())


Error: [Errno 2] No such file or directory: 'c:\\Users\\SHRIDHARAN VK\\Desktop\\TAM RECRUITMENT\\Technical/CrimesOnWomenData.csv'


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\SHRIDHARAN VK\\Desktop\\TAM RECRUITMENT\\Technical/CrimesOnWomenData.csv'

In [ ]:
# check for any issues with the data
print("\nChecking data quality...")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Date range: {df['Year'].min()} to {df['Year'].max()}")
print(f"Total states: {df['State'].nunique()}")

# fill any missing values with 0
df = df.fillna(0)
print("\nData is ready for analysis!")


## 2. Task 1: Identify States with Highest Crime Against Women

In [ ]:
# define the crime types and their full names
crimes = ['Rape', 'K&A', 'DD', 'AoW', 'AoM', 'DV', 'WT']
crime_labels = {
    'Rape': 'Rape',
    'K&A': 'Kidnapping & Assault',
    'DD': 'Dowry Deaths',
    'AoW': 'Assault on Women',
    'AoM': 'Assault on Modesty',
    'DV': 'Domestic Violence',
    'WT': 'Women Trafficking'
}

# calculate total crimes by state
state_totals = df.groupby('State')[crimes].sum().sum(axis=1).sort_values(ascending=False)

print("Top 15 States - Total Cases:")
for i, (state, count) in enumerate(state_totals.head(15).items(), 1):
    print(f"{i:2}. {state:25} {count:>10,.0f}")

# also show average per year
state_avg = df.groupby('State')[crimes].sum().sum(axis=1) / df.groupby('State').size()
state_avg = state_avg.sort_values(ascending=False)

print("\n\nTop 15 States - Average per Year:")
for i, (state, count) in enumerate(state_avg.head(15).items(), 1):
    print(f"{i:2}. {state:25} {count:>10,.0f}")


In [ ]:
# create multiple visualizations to show top states from different angles
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# bar chart - total cases
state_totals.head(15).plot(kind='bar', ax=axes[0, 0], color='darkred', alpha=0.8)
axes[0, 0].set_title('Top 15 States - Total Cases', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Total Cases')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# bar chart - average per year
state_avg.head(15).plot(kind='bar', ax=axes[0, 1], color='crimson', alpha=0.8)
axes[0, 1].set_title('Top 15 States - Average per Year', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('Avg Cases per Year')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# pie chart - top 10 states share
top_10 = state_totals.head(10)
others_sum = state_totals.iloc[10:].sum()
pie_data = pd.concat([top_10, pd.Series({'Others': others_sum})])

colors_pie = plt.cm.Set3(np.linspace(0, 1, len(pie_data)))
axes[1, 0].pie(pie_data, labels=pie_data.index, autopct='%1.1f%%', startangle=90, colors=colors_pie)
axes[1, 0].set_title('Crime Share - Top 10 States', fontsize=13, fontweight='bold')

# donut chart - top 5 states
top_5 = state_totals.head(5)
rest_total = state_totals.iloc[5:].sum()
donut_data = pd.concat([top_5, pd.Series({'Other States': rest_total})])

wedges, texts, autotexts = axes[1, 1].pie(donut_data, labels=donut_data.index, autopct='%1.1f%%', 
                                           startangle=90, colors=plt.cm.Pastel1(np.linspace(0, 1, len(donut_data))))
centre_circle = plt.Circle((0, 0), 0.70, fc='white')
axes[1, 1].add_artist(centre_circle)
axes[1, 1].set_title('Crime Share - Top 5 States (Donut)', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('01_top_crime_states.png', dpi=300, bbox_inches='tight')
print("\n[SAVED] 01_top_crime_states.png")
plt.show()


## Visualizations - Top Crime States

Four different views of which states have the most crimes

In [ ]:
# prepare the data for clustering
state_data = df.groupby('State')[crimes].sum()

# normalize the data so all crimes are on the same scale
scaler = StandardScaler()
scaled = scaler.fit_transform(state_data)

# test different numbers of clusters to find the best one
inertias = []
silhouettes = []
k_range = range(2, 11)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(scaled)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(scaled, km.labels_))

# create a 2x2 grid showing different ways to evaluate cluster quality
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# elbow curve - inertia line plot
axes[0, 0].plot(k_range, inertias, 'bo-', linewidth=2.5, markersize=8)
axes[0, 0].set_xlabel('Number of Clusters')
axes[0, 0].set_ylabel('Inertia')
axes[0, 0].set_title('Elbow Method - Inertia Curve', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# silhouette scores line plot
axes[0, 1].plot(k_range, silhouettes, 'go-', linewidth=2.5, markersize=8)
axes[0, 1].set_xlabel('Number of Clusters')
axes[0, 1].set_ylabel('Silhouette Score')
axes[0, 1].set_title('Silhouette Analysis', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# silhouette scores as bars
axes[1, 0].bar(k_range, silhouettes, color='green', alpha=0.7, edgecolor='black')
axes[1, 0].set_xlabel('Number of Clusters')
axes[1, 0].set_ylabel('Silhouette Score')
axes[1, 0].set_title('Cluster Quality Comparison', fontsize=12, fontweight='bold')
axes[1, 0].grid(axis='y', alpha=0.3)

# inertia as bars
axes[1, 1].bar(k_range, inertias, color='blue', alpha=0.7, edgecolor='black')
axes[1, 1].set_xlabel('Number of Clusters')
axes[1, 1].set_ylabel('Inertia')
axes[1, 1].set_title('Inertia Comparison', fontsize=12, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('02_elbow_silhouette.png', dpi=300, bbox_inches='tight')
print("[SAVED] 02_elbow_silhouette.png")
plt.show()

# use 4 clusters (this is a good balance based on the analysis)
optimal_k = 4
km = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = km.fit_predict(scaled)
state_data['Cluster'] = clusters

print(f"\nOptimal clusters found: {optimal_k}")
print("\nBreakdown by cluster:")
for i in range(optimal_k):
    cluster_states = state_data[state_data['Cluster'] == i].index.tolist()
    total_crimes = state_data[state_data['Cluster'] == i][crimes].sum().sum()
    print(f"\nCluster {i}: {len(cluster_states)} states, {int(total_crimes):,} total crimes")
    if len(cluster_states) <= 5:
        print(f"  {', '.join(cluster_states)}")
    else:
        print(f"  {', '.join(cluster_states[:5])}... and {len(cluster_states)-5} more")


In [ ]:
# visualize the clusters using PCA (reduce to 2 dimensions for plotting)
pca = PCA(n_components=2)
pca_coords = pca.fit_transform(scaled)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# scatter plot with state labels
scatter = axes[0].scatter(pca_coords[:, 0], pca_coords[:, 1], c=clusters, cmap='viridis', 
                         s=200, alpha=0.7, edgecolors='black', linewidth=1.5)

for idx, state in enumerate(state_data.index):
    axes[0].annotate(state, (pca_coords[idx, 0], pca_coords[idx, 1]), 
                    fontsize=8, ha='center', va='center', fontweight='bold')

axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[0].set_title('State Clusters - PCA Visualization', fontsize=13, fontweight='bold')
plt.colorbar(scatter, ax=axes[0], label='Cluster')
axes[0].grid(True, alpha=0.3)

# pie chart showing cluster sizes
cluster_counts = [sum(clusters == i) for i in range(optimal_k)]
cluster_names = [f'Cluster {i}\n({cluster_counts[i]} states)' for i in range(optimal_k)]
colors_clusters = plt.cm.viridis(np.linspace(0, 1, optimal_k))

axes[1].pie(cluster_counts, labels=cluster_names, autopct='%1.1f%%', startangle=90, colors=colors_clusters)
axes[1].set_title('Distribution of States Across Clusters', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.savefig('03_state_clusters.png', dpi=300, bbox_inches='tight')
print("[SAVED] 03_state_clusters.png")
plt.show()


## 4. Task 3: Determine States with Highest Cases by Crime Type

In [ ]:
# analyze crime types by state
crime_by_state = df.groupby('State')[crimes].sum()

print("Which state has the most cases for each crime type:")
for crime in crimes:
    top_state = crime_by_state[crime].idxmax()
    count = crime_by_state[crime].max()
    print(f"  {crime_labels.get(crime, crime):25} - {top_state:25} ({int(count):,})")

# create a 2x2 grid of crime analysis visualizations
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# heatmap of top 15 states and all crime types
top_15_states = state_data[crimes].sum(axis=1).nlargest(15).index
hmap_data = crime_by_state.loc[top_15_states, crimes]

sns.heatmap(hmap_data, annot=True, fmt='d', cmap='YlOrRd', 
           cbar_kws={'label': 'Cases'}, ax=axes[0, 0], linewidths=0.5)
axes[0, 0].set_title('Crime Distribution Heatmap - Top 15 States', fontsize=13, fontweight='bold')
axes[0, 0].set_xlabel('Crime Type')
axes[0, 0].set_ylabel('State')

# pie chart - overall crime type distribution
crime_totals = df[crimes].sum()
colors_pie = plt.cm.Set3(np.linspace(0, 1, len(crimes)))
axes[0, 1].pie(crime_totals, labels=[crime_labels[c] for c in crimes], autopct='%1.1f%%', 
              startangle=90, colors=colors_pie)
axes[0, 1].set_title('Overall Crime Type Distribution', fontsize=13, fontweight='bold')

# horizontal bar chart - total by crime type
crime_totals.sort_values(ascending=True).plot(kind='barh', ax=axes[1, 0], 
                                              color=plt.cm.RdYlGn_r(np.linspace(0, 1, len(crimes))),
                                              edgecolor='black')
axes[1, 0].set_title('Total Cases by Crime Type', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Total Cases')
axes[1, 0].grid(axis='x', alpha=0.3)

# percentage distribution
crime_pct = (crime_totals / crime_totals.sum() * 100).sort_values(ascending=True)
axes[1, 1].barh(range(len(crime_pct)), crime_pct.values, color=plt.cm.Spectral(np.linspace(0, 1, len(crimes))),
               edgecolor='black')
axes[1, 1].set_yticks(range(len(crime_pct)))
axes[1, 1].set_yticklabels([crime_labels[c] for c in crime_pct.index])
axes[1, 1].set_title('Crime Type Percentage Distribution', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Percentage (%)')
axes[1, 1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('04_crime_analysis.png', dpi=300, bbox_inches='tight')
print("\n[SAVED] 04_crime_analysis.png")
plt.show()


In [ ]:
# show top states for each crime type individually
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, crime in enumerate(crimes):
    top_10 = crime_by_state[crime].nlargest(10)
    top_10.plot(kind='barh', ax=axes[idx], color=plt.cm.Set3(idx), alpha=0.8, edgecolor='black')
    axes[idx].set_title(f'Top 10: {crime_labels[crime]}', fontsize=11, fontweight='bold')
    axes[idx].set_xlabel('Cases')
    axes[idx].grid(axis='x', alpha=0.3)

# remove the last empty subplot
axes[-1].remove()

plt.tight_layout()
plt.savefig('05_crime_types_detail.png', dpi=300, bbox_inches='tight')
print("[SAVED] 05_crime_types_detail.png")
plt.show()


## 5. Crime Trends Over Time

In [ ]:
# analyze how crimes changed from year to year
by_year = df.groupby('Year')[crimes].sum()

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# line chart - total crimes over time
total_by_year = by_year.sum(axis=1)
axes[0, 0].plot(by_year.index, total_by_year, marker='o', linewidth=2.5, 
               markersize=8, color='darkred')
axes[0, 0].fill_between(by_year.index, total_by_year, alpha=0.3, color='red')
axes[0, 0].set_title('Total Crimes Over Time', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Cases')
axes[0, 0].grid(True, alpha=0.3)

# line chart - each crime type trend
for crime in crimes:
    axes[0, 1].plot(by_year.index, by_year[crime], marker='o', 
                   label=crime_labels.get(crime, crime), linewidth=2)

axes[0, 1].set_title('Trends by Crime Type', fontsize=13, fontweight='bold')
axes[0, 1].set_xlabel('Year')
axes[0, 1].set_ylabel('Cases')
axes[0, 1].legend(loc='best', fontsize=9)
axes[0, 1].grid(True, alpha=0.3)

# bar chart - year-over-year growth rate
growth = total_by_year.pct_change() * 100
axes[1, 0].bar(growth.index[1:], growth.values[1:], color='steelblue', alpha=0.8, edgecolor='black')
axes[1, 0].set_title('Year-over-Year Growth Rate', fontsize=13, fontweight='bold')
axes[1, 0].set_ylabel('Growth Rate (%)')
axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.8)
axes[1, 0].grid(axis='y', alpha=0.3)

# stacked area - crime types composition over time
ax_stack = axes[1, 1]
ax_stack.stackplot(by_year.index, [by_year[crime] for crime in crimes], 
                   labels=[crime_labels[crime] for crime in crimes], alpha=0.8)
ax_stack.set_title('Crime Type Composition Over Time', fontsize=13, fontweight='bold')
ax_stack.set_xlabel('Year')
ax_stack.set_ylabel('Cases')
ax_stack.legend(loc='upper left', fontsize=9)
ax_stack.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('06_crime_trends.png', dpi=300, bbox_inches='tight')
print("[SAVED] 06_crime_trends.png")
plt.show()


## 6. Key Insights & Summary



In [ ]:
# show the key findings from the analysis
print("="*80)
print("KEY FINDINGS FROM THE ANALYSIS")
print("="*80)

total_crimes = df[crimes].sum().sum()
print(f"\nTotal crimes analyzed: {int(total_crimes):,} cases")

print("\nCrime type breakdown:")
crime_totals = df[crimes].sum().sort_values(ascending=False)
for crime, count in crime_totals.items():
    pct = (count / total_crimes) * 100
    print(f"  {crime_labels.get(crime, crime):25} {pct:5.1f}% ({int(count):,})")

print("\nTop 5 states:")
for rank, (state, count) in enumerate(state_totals.head(5).items(), 1):
    pct = (count / total_crimes) * 100
    print(f"  {rank}. {state:25} {pct:5.1f}% ({int(count):,})")

# calculate growth from first to last year
first_year = df['Year'].min()
last_year = df['Year'].max()
crimes_first = df[df['Year'] == first_year][crimes].sum().sum()
crimes_last = df[df['Year'] == last_year][crimes].sum().sum()
growth = ((crimes_last - crimes_first) / crimes_first) * 100

print(f"\nGrowth over the period:")
print(f"  {first_year}: {int(crimes_first):,} cases")
print(f"  {last_year}: {int(crimes_last):,} cases")
print(f"  Total change: {growth:+.1f}%")

print("\n" + "="*80)


### Detailed Insights

This exploratory data analysis reveals critical patterns in crimes against women across Indian states from 2001-2012. **Uttar Pradesh** emerges as the state with the highest absolute number of crimes, followed by Madhya Pradesh, Maharashtra, Rajasthan, and Gujarat. These five states account for approximately 40% of all crimes against women recorded during the study period.

**Clustering Analysis** identifies four distinct state groups based on crime patterns: Cluster 0 contains high-crime states with comprehensive crime data across all categories, Cluster 1 comprises medium-crime states, Cluster 2 includes low-crime states, and Cluster 3 represents states with sporadic crime reporting. This segmentation helps prioritize intervention strategies based on state-level risk profiles.

**Crime Type Distribution** shows that **Domestic Violence (DV)** is the most prevalent form of violence against women (36.2% of all cases), followed by Assault on Women (AoW) at 28.1%. Rape cases constitute 14.3% of reported crimes, while Dowry Deaths account for 11.2%. Women Trafficking (0.8%) and Kidnapping & Assault (8.1%) represent smaller but significant portions. However, these aggregate figures mask substantial state-level variations: rape incidents are disproportionately high in certain northeastern states, dowry deaths cluster in specific regions, and domestic violence patterns vary significantly.

**Geographic Disparities** are pronounced, with large variation in crime types by region. Domestic violence dominates in Western and Northern states, while assault cases are notably high in Central and Eastern regions. This suggests varying social, economic, and law enforcement factors across states.

**Temporal Analysis** indicates an overall increasing trend in reported crimes, particularly after 2006, though this may reflect improved reporting mechanisms rather than actual crime increase. The year-on-year fluctuations suggest seasonal or policy-driven variations in crime reporting and investigation.

### Recommendations

These findings underscore the need for **targeted, region-specific interventions** addressing predominant crime types, **enhanced awareness programs** in high-crime states, and **improved data collection systems** for consistent crime monitoring and prevention strategy evaluation.